# Small cookbook for manipulating pandas dataframes

In [1]:
import pandas as pd
import numpy as np

In [2]:
X = pd.DataFrame([[1,2,3],[4,5,6],['duck','duck','goose'],[0,0,0]])
X.columns = ['col1','col2','col3']
X

,col1,col2,col3
0,1,2,3
1,4,5,6
2,duck,duck,goose
3,0,0,0


In [3]:
def last_to_first(df):
    cols = list(df)
    cols.insert(0, cols.pop(cols.index(cols[-1])))
    return df.ix[:, cols]

# last_to_first(X)

In [4]:
def first_to_last(df):
    cols = list(df)
    cols.append(cols.pop(cols.index(cols[0])))
    return df.ix[:, cols]

# first_to_last(X)

In [5]:
def reorder_cols(df, cols):
    return df.ix[:, cols]

# reorder_cols(X,['col2','col3','col1'])

In [6]:
def return_every_n(df, n):
    return df.iloc[:, ::n]

# Y = pd.concat([X,X,X,X,X],axis=1)
# return_every_n(Y,2)

In [7]:
def delete_list(df, todelete):
    return df.drop(df.ix[:,todelete].head(0).columns, axis=1)

# delete_list(Y, 'col3')

In [9]:
def delete_list_ix(df, todelete):
    colnames = df.columns
    for tod in todelete:
        colnames.remove(tod)
    df.columns = range(0,df.shape[1])
    dx = df.drop(df.ix[:,todelete],axis=1)
    dx.columns = colnames
    return dx
# delete_list_ix(Y, [2,3])

In [10]:
todelete = [1,2]
columns = range(0,Y.shape[1])
# [columns.remove(tod) for tod in todelete]

NameError: name 'Y' is not defined

In [ ]:
cds = '/projects/ps-yeolab/genomes/hg19/hg19.introns.bed.txt'
bedhead = ['chrom','start','end','name','score','strand']
def get_name(row):
    """
    cdsmeans['ENST00000237247.6_cds_2_0_chr1_67091530_f']
    """
    return row['name'].split('_')[0]
cdsdf = pd.read_table(cds,names=bedhead)
cdsdf['name2'] = cdsdf.apply(get_name,axis=1)
cdsdf.set_index('name2',inplace=True)


In [ ]:
Y = pd.read_table('/home/bay001/projects/parp13_ago2_20160606/analysis/clips/clips_20160820/input_norm_manifest.txt')

In [ ]:
Y.ix[0]['CLIP']

In [13]:
# using lambda and apply
bedhead = ['chrom','start','end','name','score','strand']
df = pd.read_table(
    '/home/bay001/projects/emilie_rnae_20161003/analysis/editing/Hundley22/results/GSF860-Hundley-22_S1_R1_001.polyATrim.adapterTrim.rmRep.sorted.rg.fwd.sorted.rmdup.readfiltered.formatted.varfiltered.snpfiltered.ranked.bed',
    names=bedhead
)
df = df.head()
df['editfrac'] = df['name'].apply(lambda x: x.split('|')[2])
df

,chrom,start,end,name,score,strand,editfrac
0,chr1,322098,322099,31|A>G|0.032258065,0.739700,+,0.032258065
1,chr1,324319,324320,36|A>G|0.138888889,0.999975,+,0.138888889
2,chr1,324331,324332,25|A>G|0.44,1.000000,+,0.44
3,chr1,325513,325514,28|A>G|0.035714286,0.762343,+,0.035714286
4,chr1,325671,325672,36|A>G|0.027777778,0.703448,+,0.027777778


In [12]:
# finding all rows with a nan
df = pd.DataFrame([range(3), [0, np.NaN, 0], [0, 0, np.NaN], range(3), range(3)])
df[df.isnull().any(axis=1)]

,0,1,2
1,0,NaN,0.0
2,0,0.0,NaN
